In [2]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
from datetime import datetime as dt
import re
from matplotlib import pyplot as plt
import matplotlib
import fastText
import html
import string
import nltk
from multiprocessing import Process
import multiprocessing
import time
import sys
from importlib import reload
import redis
# from urllib import request
from urllib.parse import urlparse
import requests as req
# from lxml import html
import logging
# from scrapy.http import Request
import requests
import json
import requests
from scrapy.http import TextResponse
from bs4 import BeautifulSoup
import re
from collections import Counter
from sklearn.metrics import precision_recall_curve
from tabulate import tabulate
from nltk.metrics import ConfusionMatrix
from sklearn.metrics.pairwise import cosine_similarity
import pickle
from gensim.models import Word2Vec
import mpld3
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer


mpld3.enable_notebook()
%matplotlib inline

In [157]:
df_comparisons = pd.read_csv('df_comparisons_unsup_lemma.csv', index_col=0)
df_news = pd.read_csv('df_news_unsup_lemma.csv',  parse_dates=['Datetime', 'Date'], index_col=0)

In [253]:
df_news.spot_pp_str.fillna('', inplace = True)
df_news.text_pp_str.fillna('', inplace = True)
df_news.title_pp_str.fillna('', inplace = True)

In [159]:
def get_freq_unique(textnow):
    
    counts = Counter(textnow.split())

    wordlist = []
    countlist = []
    for word, count in counts.items():
        wordlist.append(word)
        countlist.append(count)
    countlist = np.array(countlist)
#     countlist = countlist / countlist.sum()
    
    return pd.Series({'words_unique': wordlist, 'words_count_unique': countlist})

In [249]:
methods = ['wmd_ft', 'wmd_wv']
# methods = ['ju','js','jc','ft']
fields = ['title', 'spot', 'text']
methods_cross = [met + '_' + field for met in methods for field in fields]

In [254]:
for field in fields:
    df_tmp = df_news[field + '_pp_str'].apply(get_freq_unique)
    df_news['words_unique_' + field] = df_tmp['words_unique']
    df_news['words_count_unique_' + field] = df_tmp['words_count_unique']

In [59]:
ftmodel = fastText.load_model('../../data/model/downloads/haber-P1_S1_L1-100E.bin')
model_w2v = Word2Vec.load('../../data/model/downloads/reaction.all.1544173485.punct_True.stopwords_True.lemmatized_True.w2v_100V_5E_1547415130.bin')

In [347]:
def get_w2v(wordlist):
    
    tmp = [(index, word) for index, word in enumerate(wordlist) if word in model_w2v.wv.vocab]
    
    if len(tmp) == 0:
        return None, None
    
    indexes_in, words_in = list(zip(*tmp))
    
    count = len(words_in)
    
    X = np.zeros((count, model_w2v.wv.vector_size))
    
    for i, word in enumerate(words_in):
        X[i,:] = model_w2v.wv.word_vec(word)
    
    return X, np.array(indexes_in)

In [325]:
words1 = row1['words_unique_'+field]
words2 = row2['words_unique_'+field]

In [332]:
X1 = np.concatenate([ftmodel.get_word_vector(word).reshape(1,-1) for word in words1], axis=0)
X2 = np.concatenate([ftmodel.get_word_vector(word).reshape(1,-1) for word in words2], axis=0)

In [334]:
X1.shape

(41, 100)

In [349]:
for field in fields:
    
    print('\n\n'+field+'\t')
    
    for index, row in df_comparisons.iterrows():
        
        if index % 1000 == 0:
            print(index, end=',')
#         print(index)
        # JU, JS and JC scores
        row1 = df_news.iloc[int(row.ind0)]
        row2 = df_news.iloc[int(row.ind1)]
        
        words1 = row1['words_unique_'+field]
        words2 = row2['words_unique_'+field]
        
        if len(words1) == 0 or len(words2) == 0:
            df_comparisons.loc[index, 'wmd_wv_'+field] = None
            df_comparisons.loc[index, 'wmd_ft_'+field] = None
            continue
            
            
        # Fasttext
            
        X1 = np.concatenate([ftmodel.get_word_vector(word).reshape(1,-1) for word in words1], axis=0)
        X2 = np.concatenate([ftmodel.get_word_vector(word).reshape(1,-1) for word in words2], axis=0)
        
        d1 = row1['words_count_unique_'+field]
        d2 = row2['words_count_unique_'+field]
        
        d1 = d1 / d1.sum()
        d2 = d2 / d2.sum()

        C = np.sqrt(((np.expand_dims(X1, 1) - np.expand_dims(X2, 0)) ** 2).sum(axis=-1))

        T1 = np.zeros_like(C)
        T1[np.arange(T1.shape[0]), C.argmin(axis=1)] = d1
        T2 = np.zeros_like(C)
        T2[C.argmin(axis=0), np.arange(T2.shape[1])] = d2

        dist = max(np.sum(T1 * C), np.sum(T2 * C))
        
        df_comparisons.loc[index, 'wmd_ft_'+field] = dist
        
        
        # Word2Vec
        
        X1, indexes1 = get_w2v(words1)
        X2, indexes2 = get_w2v(words2)
        
        if X1 is None or X2 is None:
            df_comparisons.loc[index, 'wmd_wv_'+field] = None
            continue
        
        d1 = row1['words_count_unique_'+field][indexes1]
        d2 = row2['words_count_unique_'+field][indexes2]
        
        d1 = d1 / d1.sum()
        d2 = d2 / d2.sum()

        C = np.sqrt(((np.expand_dims(X1, 1) - np.expand_dims(X2, 0)) ** 2).sum(axis=-1))

        T1 = np.zeros_like(C)
        T1[np.arange(T1.shape[0]), C.argmin(axis=1)] = d1
        T2 = np.zeros_like(C)
        T2[C.argmin(axis=0), np.arange(T2.shape[1])] = d2

        dist = max(np.sum(T1 * C), np.sum(T2 * C))
        
        df_comparisons.loc[index, 'wmd_wv_'+field] = dist



title	
0,1000,2000,3000,4000,5000,6000,7000,8000,9000,10000,11000,12000,13000,14000,15000,16000,

spot	
0,1000,2000,3000,4000,5000,6000,7000,8000,9000,10000,11000,12000,13000,14000,15000,16000,

text	
0,1000,2000,3000,4000,5000,6000,7000,8000,9000,10000,11000,12000,13000,14000,15000,16000,

In [350]:
df_comparisons_bu = df_comparisons.copy()

In [351]:
for met in methods:
    for field in fields:
        df_comparisons[met+'_'+field] = df_comparisons[met+'_'+field].max() - df_comparisons[met+'_'+field]

In [352]:
for met in methods:
    for field in fields:
        colnow = met+'_'+field
        missings_now = df_comparisons_bu[colnow].isnull()
        meannow = df_comparisons_bu[~missings_now][colnow].mean()
        df_comparisons.loc[missings_now, colnow] = meannow
        print(colnow, missings_now.sum(), meannow)

wmd_ft_title 913 3.26193613620173
wmd_ft_spot 241 2.957422562977026
wmd_ft_text 0 2.612090747701467
wmd_wv_title 913 3.3672491963278097
wmd_wv_spot 241 3.109255528833871
wmd_wv_text 0 2.709321780494125


In [353]:
for met in methods:
    for field in fields:
        colnow = met+'_'+field
        missings_now = df_comparisons[colnow].isnull()
        meannow = df_comparisons[~missings_now][colnow].mean()
        print(colnow, missings_now.sum(), meannow)

wmd_ft_title 0 2.714834635639943
wmd_ft_spot 0 3.1387388889676076
wmd_ft_text 0 3.0158003159887183
wmd_wv_title 0 2.677124345675647
wmd_wv_spot 0 1.7750465242013165
wmd_wv_text 0 1.9120923118631081


In [354]:
def prec_recall(tp, fp, fn):

    return float(tp / (tp + fp)), float(tp / (tp + fn))

def find_eacc(labels_now, preds_now):
    labels_now = labels_now.flatten()
    preds_now = preds_now.flatten()
    
    precs, recalls, thresholds = precision_recall_curve(labels_now, preds_now)
    
    indnow = np.argmin(np.abs(precs-recalls))
    eacc = (precs[indnow] + recalls[indnow]) / 2
    thr = thresholds[indnow]
    
    true_positive = np.logical_and(preds_now>=thr, labels_now==1).sum()
    true_negative = np.logical_and(preds_now<thr, labels_now==0).sum()
    false_positive = np.logical_and(preds_now>=thr, labels_now==0).sum()
    false_negative = np.logical_and(preds_now<thr, labels_now==1).sum()
    
    precision, recall = prec_recall(true_positive, false_positive, false_negative)
    
    f1 = 2*(recall * precision) / (recall + precision)
    
    out = dict(thr=thr,eacc=eacc, precision=precision, recall=recall, f1=f1,
              tp=true_positive, tn=true_negative,
               fp=false_positive, fn=false_negative,
               precs=precs, recalls=recalls
               )
    
    return out

In [367]:
results = {}
for met in methods:
    for field in fields:
        method = met + '_' + field
        results[method] = find_eacc(df_comparisons.label.values, df_comparisons[method].values) 

In [368]:
def tabulate_res(resnow):

    table = []
    for key, val in resnow.items():
        table.append(
            [key, resnow[key]['eacc'], resnow[key]['f1'], resnow[key]['precision'], resnow[key]['recall'], resnow[key]['thr'], resnow[key]['tp'], resnow[key]['tn'], resnow[key]['fp'],
             resnow[key]['fn']])

    return tabulate(table, headers=['', 'E-ACC', 'F1', 'PREC', 'RECALL', 'THR', 'TP', 'TN', 'FP', 'FN'], tablefmt="fancy_grid")

In [369]:
print(tabulate_res(results))

╒══════════════╤══════════╤══════════╤══════════╤══════════╤═════════╤══════╤═══════╤══════╤══════╕
│              │    E-ACC │       F1 │     PREC │   RECALL │     THR │   TP │    TN │   FP │   FN │
╞══════════════╪══════════╪══════════╪══════════╪══════════╪═════════╪══════╪═══════╪══════╪══════╡
│ wmd_ft_title │ 0.829925 │ 0.829925 │ 0.829925 │ 0.829925 │ 3.31806 │ 1542 │ 14684 │  316 │  316 │
├──────────────┼──────────┼──────────┼──────────┼──────────┼─────────┼──────┼───────┼──────┼──────┤
│ wmd_ft_spot  │ 0.938644 │ 0.938644 │ 0.938644 │ 0.938644 │ 3.59613 │ 1744 │ 14886 │  114 │  114 │
├──────────────┼──────────┼──────────┼──────────┼──────────┼─────────┼──────┼───────┼──────┼──────┤
│ wmd_ft_text  │ 0.926265 │ 0.926265 │ 0.926265 │ 0.926265 │ 3.50614 │ 1721 │ 14863 │  137 │  137 │
├──────────────┼──────────┼──────────┼──────────┼──────────┼─────────┼──────┼───────┼──────┼──────┤
│ wmd_wv_title │ 0.8485   │ 0.846501 │ 0.88968  │ 0.80732  │ 3.36949 │ 1500 │ 14814 │  186 │  358 │


In [370]:
def normalize_score(dfnow, colname, thrnow):
    
    stdnow = dfnow[colname].std()
    
    print("%s\t==>\tstd: %.4f, old_thr: %.4f" % (colname, stdnow, thrnow))
    
    return (dfnow[colname] - thrnow ) / stdnow, stdnow

In [371]:
methods_cross

['wmd_ft_title',
 'wmd_ft_spot',
 'wmd_ft_text',
 'wmd_wv_title',
 'wmd_wv_spot',
 'wmd_wv_text']

In [372]:
for method in methods_cross:
    dsnow, stdnow = normalize_score(df_comparisons, method, results[method]['thr'])
    df_comparisons[method+'_norm'] = dsnow
    results[method]['std_norm'] = stdnow

wmd_ft_title	==>	std: 0.7643, old_thr: 3.3181
wmd_ft_spot	==>	std: 0.7576, old_thr: 3.5961
wmd_ft_text	==>	std: 0.7648, old_thr: 3.5061
wmd_wv_title	==>	std: 0.7608, old_thr: 3.3695
wmd_wv_spot	==>	std: 0.7788, old_thr: 2.6212
wmd_wv_text	==>	std: 0.7364, old_thr: 2.4242


In [373]:
methods_cross_norm = [met+'_norm' for met in methods_cross]

In [374]:
field_weights = {'spot': 0.3, 'title': 0.1, 'text': 0.6}

In [375]:
def weighted_av(rownow, weights_field, metsnow):
    
    score = 0
    count = 0
    
    for method in metsnow:
        count += 1
        for field in fields:
            score += weights_field[field] * rownow[method + '_' + field + '_norm']
            
    return score / count

In [376]:
for met in methods:
    df_comparisons[met + '_all'] = df_comparisons.apply(lambda row: weighted_av(row, field_weights, [met]), axis=1)
    results.update({met + '_all': find_eacc(df_comparisons.label.values, df_comparisons[met + '_all'].values)})

In [377]:
results.update({met+'_all': find_eacc(df_comparisons.label.values, df_comparisons[met+'_all'].values) for met in methods})

In [378]:
print(tabulate_res(results))

╒══════════════╤══════════╤══════════╤══════════╤══════════╤═════════════╤══════╤═══════╤══════╤══════╕
│              │    E-ACC │       F1 │     PREC │   RECALL │         THR │   TP │    TN │   FP │   FN │
╞══════════════╪══════════╪══════════╪══════════╪══════════╪═════════════╪══════╪═══════╪══════╪══════╡
│ wmd_ft_title │ 0.829925 │ 0.829925 │ 0.829925 │ 0.829925 │  3.31806    │ 1542 │ 14684 │  316 │  316 │
├──────────────┼──────────┼──────────┼──────────┼──────────┼─────────────┼──────┼───────┼──────┼──────┤
│ wmd_ft_spot  │ 0.938644 │ 0.938644 │ 0.938644 │ 0.938644 │  3.59613    │ 1744 │ 14886 │  114 │  114 │
├──────────────┼──────────┼──────────┼──────────┼──────────┼─────────────┼──────┼───────┼──────┼──────┤
│ wmd_ft_text  │ 0.926265 │ 0.926265 │ 0.926265 │ 0.926265 │  3.50614    │ 1721 │ 14863 │  137 │  137 │
├──────────────┼──────────┼──────────┼──────────┼──────────┼─────────────┼──────┼───────┼──────┼──────┤
│ wmd_wv_title │ 0.8485   │ 0.846501 │ 0.88968  │ 0.80732  │  3.

In [379]:
df_comparisons.to_csv('df_comparisons_unsup_lemma_wmd.csv')

In [380]:
with open('unsupervised_results_lemma_wmd.pickle', 'wb') as f:
    pickle.dump(results, f)